# 6 AutoGen - Human interaction

AutoGen is a framework that enables the development of LLM applications using multiple agents that can converse with each other to solve tasks. AutoGen agents are customizable, conversable, and seamlessly allow human participation. They can operate in various modes that employ combinations of LLMs, human inputs, and tools.

17/06/2024

<img src="https://raw.githubusercontent.com/retkowsky/images/master/autogenlogo.jpg">

> Documentation: https://microsoft.github.io/autogen/

In [1]:
import autogen
import json
import openai
import os
import sys

from autogen import config_list_from_json

In [2]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    #filter_dict={"tags": ["gpt-4"]},  # comment out to get all
)
# When using a single openai endpoint, you can use the following:
# config_list = [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]

In [4]:
WORK_DIR = "workdir"

os.makedirs(WORK_DIR, exist_ok=True)

## 1. Agent

In [5]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

In [6]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
    },
)

In [7]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith(
        "TERMINATE"),
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code.
)

In [8]:
problem_to_solve = """
I want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.
Show me the python code.
"""

In [9]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(assistant, message=problem_to_solve)

user_proxy (to assistant):


I want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.
Show me the python code.


--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, you can use Pillow, a Python Imaging Library to accomplish your task. 

Here's the code:

```python
# filename: resize_images.py

import os
from PIL import Image

def resize_images():
    directory = 'MYIMAGES'
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg"): 
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
            img = img.resize((640, 640))
            img.save(img_path)

resize_images()
```

Steps to Execute:
1. Copy the code into a python file `resize_images.py`.
2. Replace `'MYIMAGES'` with your image directory path if it is different.
3. Run the python file.

This code will resize all images in the given directory t

ChatResult(chat_id=None, chat_history=[{'content': '\nI want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.\nShow me the python code.\n', 'role': 'assistant'}, {'content': 'Sure, you can use Pillow, a Python Imaging Library to accomplish your task. \n\nHere\'s the code:\n\n```python\n# filename: resize_images.py\n\nimport os\nfrom PIL import Image\n\ndef resize_images():\n    directory = \'MYIMAGES\'\n    for filename in os.listdir(directory):\n        if filename.endswith(".jpeg") or filename.endswith(".jpg"): \n            img_path = os.path.join(directory, filename)\n            img = Image.open(img_path)\n            img = img.resize((640, 640))\n            img.save(img_path)\n\nresize_images()\n```\n\nSteps to Execute:\n1. Copy the code into a python file `resize_images.py`.\n2. Replace `\'MYIMAGES\'` with your image directory path if it is different.\n3. Run the python file.\n\nThis code will resize all images in the given directory to 640x640 pixe

> You can interact and say for example to use another library like OpenCV

In [10]:
print(user_proxy.chat_messages[assistant])

[{'content': '\nI want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.\nShow me the python code.\n', 'role': 'assistant'}, {'content': 'Sure, you can use Pillow, a Python Imaging Library to accomplish your task. \n\nHere\'s the code:\n\n```python\n# filename: resize_images.py\n\nimport os\nfrom PIL import Image\n\ndef resize_images():\n    directory = \'MYIMAGES\'\n    for filename in os.listdir(directory):\n        if filename.endswith(".jpeg") or filename.endswith(".jpg"): \n            img_path = os.path.join(directory, filename)\n            img = Image.open(img_path)\n            img = img.resize((640, 640))\n            img.save(img_path)\n\nresize_images()\n```\n\nSteps to Execute:\n1. Copy the code into a python file `resize_images.py`.\n2. Replace `\'MYIMAGES\'` with your image directory path if it is different.\n3. Run the python file.\n\nThis code will resize all images in the given directory to 640x640 pixels. Please do back up your images befo

In [11]:
outputfile = os.path.join(WORK_DIR, "human_interaction.json")

json.dump(user_proxy.chat_messages[assistant], open(outputfile, "w"), indent=6)

In [12]:
print("\033[1;31;34m")
with open(outputfile, "r") as f:
    print(f.read())


[
      {
            "content": "\nI want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.\nShow me the python code.\n",
            "role": "assistant"
      },
      {
            "content": "Sure, you can use Pillow, a Python Imaging Library to accomplish your task. \n\nHere's the code:\n\n```python\n# filename: resize_images.py\n\nimport os\nfrom PIL import Image\n\ndef resize_images():\n    directory = 'MYIMAGES'\n    for filename in os.listdir(directory):\n        if filename.endswith(\".jpeg\") or filename.endswith(\".jpg\"): \n            img_path = os.path.join(directory, filename)\n            img = Image.open(img_path)\n            img = img.resize((640, 640))\n            img.save(img_path)\n\nresize_images()\n```\n\nSteps to Execute:\n1. Copy the code into a python file `resize_images.py`.\n2. Replace `'MYIMAGES'` with your image directory path if it is different.\n3. Run the python file.\n\nThis code will resize all images in the given direc

In [13]:
!ls $WORK_DIR/*.json -lt

-rwxrwxrwx 1 root root 3787 Jun 19 08:49 workdir/human_interaction.json
